In [0]:
%run "./reader_factory"

In [0]:
%run "./transform"

In [0]:
%run "./extractor"

In [0]:
dbutils.fs.put("/FileStore/sink_data/hello.txt", "Hello, world!", True)


Wrote 13 bytes.
Out[10]: True

In [0]:
dbutils.fs.rm("/FileStore/sink_data/hello.txt")


Out[11]: True

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/OnlyAirpodsAndIphone")


Out[64]: True

In [0]:
%run "./loader_factory"

In [0]:
%run "./loader"

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

In [0]:
spark = SparkSession.builder.appName("thebigdatashow.me").getOrCreate()
input_df = spark.read.format('csv').option('header',True).load("dbfs:/FileStore/tables/Customer_Updated.csv")
input_df.display()

customer_id,customer_name,join_date,location
105,Eva,2022-01-01,Ohio
106,Frank,2022-02-01,Nevada
107,Grace,2022-03-01,Colorado
108,Henry,2022-04-01,Utah


In [0]:
class FirstWorkflow:
    """ 
    ETL Pipeline to generate the data for all customer who bought airpods just after buying iphone
    """


    def __init__(self):
        pass
    def runner(self):
        # step 1 :- extract all req data from different source

        inputDFs = Airpodsafteriphoneextractor().extract()

        # step2 :- implement the transformation logic
        # people buying airpods after buying iphone

        # calling Firsttransformer class and from that class transform method 
        # file name of this class = transform.ipynb
        first_transformed_df = AirpodsAfterIphone().transform(inputDFs)

        # load the all the data to different sink

        Airpodafteriphoneloader(first_transformed_df).sink()

        
         

        # transaction_input_df.display()

# firstworkflow = FirstWorkflow()
# firstworkflow.runner()

# lead() partition by customer_id & order by transaction date

In [0]:
class SecondWorkflow:
    """ 
    ETL Pipeline to generate the data for all customer who bought only airpods and iphone
    """


    def __init__(self):
        pass
    def runner(self):
        # step 1 :- extract all req data from different source

        inputDFs = Airpodsafteriphoneextractor().extract()

        # step2 :- implement the transformation logic
        # people buying airpods after buying iphone

        # calling Firsttransformer class and from that class transform method 
        # file name of this class = transform.ipynb
        only_airpods_and_iphonedf = OnlyAirpodsAndIphone().transform(inputDFs)

        # load the all the data to different sink

        OnlyAirpodsAndIphoneloader(only_airpods_and_iphonedf).sink()

        
         

        # transaction_input_df.display()

# secondworkflow = SecondWorkflow()
# secondworkflow.runner()

# lead() partition by customer_id & order by transaction date

In [0]:
dbutils.fs.rm("dbfs:/FileStore/sink_data/OnlyAirpodsAndIphone", recurse=True)


Out[69]: True

In [0]:
class WorkFlow:
    def __init__(self,name):
        self.name = name

    def runner(self):
        if self.name == "FirstWorkflow":
            return FirstWorkflow().runner()
        elif self.name == "SecondWorkflow":
            return SecondWorkflow().runner()
    
name = "SecondWorkflow"
workflowrunner = WorkFlow(name).runner()

transaction_id,customer_id,product_name,transaction_date
11,105,iPhone,2022-02-01
14,105,AirPods,2022-02-04
18,105,MacBook,2022-02-08
12,106,iPhone,2022-02-02
16,106,MacBook,2022-02-06
20,106,AirPods,2022-02-10
13,107,AirPods,2022-02-03
17,107,iPhone,2022-02-07
15,108,iPhone,2022-02-05
19,108,AirPods,2022-02-09


customer table !!!!


customer_id,customer_name,join_date,location
105,Eva,2022-01-01,Ohio
106,Frank,2022-02-01,Nevada
107,Grace,2022-03-01,Colorado
108,Henry,2022-04-01,Utah


transaction_input_df in transform


transaction_id,customer_id,product_name,transaction_date
11,105,iPhone,2022-02-01
12,106,iPhone,2022-02-02
13,107,AirPods,2022-02-03
14,105,AirPods,2022-02-04
15,108,iPhone,2022-02-05
16,106,MacBook,2022-02-06
17,107,iPhone,2022-02-07
18,105,MacBook,2022-02-08
19,108,AirPods,2022-02-09
20,106,AirPods,2022-02-10


customer_id,produts
107,"List(AirPods, iPhone)"
108,"List(AirPods, iPhone)"
106,"List(AirPods, iPhone, MacBook)"
105,"List(AirPods, iPhone, MacBook)"


customer_id,produts
107,"List(AirPods, iPhone)"
108,"List(AirPods, iPhone)"


join df !!!!!


customer_id,customer_name,join_date,location,produts
107,Grace,2022-03-01,Colorado,"List(AirPods, iPhone)"
108,Henry,2022-04-01,Utah,"List(AirPods, iPhone)"
